In [1]:
import os
import pandas as pd
from GraphBuild import * 
import matplotlib.pyplot as plt
import random
import pickle

In [2]:
# Initialize Circuit
FolderName = os.path.dirname(os.path.realpath("__file__"))
DSSfile = r""+ FolderName+ "\IEEE123Master.dss"
Ckt_obj = CircuitSetup(DSSfile)  #creating a DSS object instance

In [3]:
G_original =  build_graph(Ckt_obj)
nx.readwrite.gml.write_gml(G_original,"123busEx.gml") #Graph undirected with edge features and node features which are constant
node_list=list(G_original.nodes())
edge_list=list(G_original.edges())

In [20]:
node_list[0]

'150'

In [21]:
nbus_sensors = math.ceil(len(node_list)*0.4)
nbranch_sensors = math.ceil(len(edge_list)*0.4)

In [27]:
source_bus = node_list[0]
bus_sensors = random.sample(node_list,nbus_sensors)
if source_bus not in bus_sensors:
    bus_sensors = random.sample(node_list,nbus_sensors)
else:
    print(bus_sensors)

['18', '81', '3', '17', '77', '152', '27', '9', '150', '49', '610', '5', '46', '85', '38', '8', '7', '57', '99', '160r', '160', '113', '42', '50', '45', '97', '22', '30', '96', '108', '114', '95', '74', '61s', '16', '65', '92', '26', '13', '2', '20', '37', '25r', '55', '1', '76', '100', '87', '25', '63', '48', '11', '300_open']


In [28]:
with open('bus_sensors.pkl', 'wb') as f:
    pickle.dump(bus_sensors, f)

In [33]:
source_branch = edge_list[0]
branch_sensors = random.sample(edge_list,nbranch_sensors)
if source_branch not in branch_sensors:
    branch_sensors = random.sample(edge_list,nbranch_sensors)
else:
    print(branch_sensors)

[('8', '12'), ('81', '84'), ('13', '152'), ('5', '6'), ('51', '151'), ('21', '23'), ('34', '15'), ('57', '58'), ('60', '61'), ('23', '24'), ('160r', '160'), ('18', '19'), ('52', '152'), ('87', '88'), ('35', '135'), ('61', '61s'), ('101', '105'), ('49', '50'), ('73', '74'), ('30', '250'), ('26', '27'), ('78', '80'), ('110', '112'), ('40', '41'), ('23', '25'), ('89', '91'), ('97', '197'), ('14', '10'), ('9r', '14'), ('25', '25r'), ('150r', '149'), ('35', '36'), ('58', '59'), ('9', '9r'), ('52', '53'), ('67', '160r'), ('110', '111'), ('27', '33'), ('95', '96'), ('26', '31'), ('13', '34'), ('69', '70'), ('150', '150r'), ('77', '78'), ('102', '103'), ('36', '38'), ('1', '2'), ('42', '44'), ('78', '79'), ('57', '60'), ('93', '95'), ('62', '63'), ('36', '37')]


In [34]:
with open('branch_sensors.pkl', 'wb') as f:
    pickle.dump(branch_sensors, f)

In [4]:
input_file = 'LoadShape1.xlsx'
sheet_name = 'LoadShape1'  # Change to the sheet you want to split
df = pd.read_excel(input_file, sheet_name=sheet_name)

points_per_day = 24  # Assuming 1-hour resolution (24 points = 24 hours)
# Convert to flat list
loadshape_values = df.iloc[:, 0].values.tolist()
# Check total number of complete days available
num_days = len(loadshape_values) // points_per_day
# Slice the loadshape into daily
LoadShapes = []
for i in range(num_days):
    start_idx = i * points_per_day
    end_idx = (i + 1) * points_per_day
    daily_shape = loadshape_values[start_idx:end_idx]
    LoadShapes.append(daily_shape)

In [5]:
Scenarios  = []
scid = 0
### Scenario generation
NSc = 1 # parameter indicating no.of scenarios for each

### Select random loadshape and a multiplier and Evaluate the circuit at the snapshot

In [249]:
Ckt_obj = CircuitSetup(DSSfile)

In [8]:
loadshape_day = random.choice(LoadShapes)

In [9]:
ldmult = random.choice(loadshape_day)

In [10]:
ldmult 

0.589268250260193

In [11]:
Ckt_obj.dss.Solution.LoadMult(ldmult)
Ckt_obj.dss.Solution.Solve()

In [237]:
MAX_RADIUS_FRAC = 1 / 3
MAX_OUTAGE_PERC = 0.4

In [271]:
# Function to generate outage scenario
def generate_outage_edges(G, max_rad_frac=MAX_RADIUS_FRAC, max_percfail=MAX_OUTAGE_PERC):
    nd = random.choice(list(G.nodes()))
    max_rad = nx.diameter(G)
    rad = math.ceil(random.uniform(1, max_rad * max_rad_frac))  # ensure radius >= 1
    Gsub = nx.ego_graph(G, nd, radius=rad, undirected=False)
    sub_edges = list(Gsub.edges())
    if not sub_edges:
        return generate_outage_edges(G)  # Retry if no edges in subgraph
    out_perc = random.uniform(0.1, max_percfail)
    N_out = max(1, math.ceil(len(sub_edges) * out_perc))
    out_edges = random.sample(sub_edges, k=N_out)
    return out_edges

In [272]:
outage_edges = generate_outage_edges(G_original)
outage_lines = []
outage_types = []

In [273]:
for from_node, to_node in outage_edges:
    edge_data = G_original.get_edge_data(from_node, to_node)
    outageelem = f"{edge_data['Device']}.{edge_data['Label']}"
    if edge_data['Device'] == 'line':
        outage_lines.append(outageelem)
        Ckt_obj.dss.Circuit.SetActiveElement(outageelem)
        active_phases = Ckt_obj.dss.CktElement.NodeOrder()[:len(Ckt_obj.dss.CktElement.NodeOrder())//2]
        phase_indic = random.randint(1,len(active_phases))
        if phase_indic == 3: #3 phase
                Ckt_obj.dss.CktElement.Open(1,0)
                outage_types.append('3 ph')                
        if phase_indic == 1: #1 phase
                phx = random.choice(active_phases)
                Ckt_obj.dss.CktElement.Open(1,phx)
                outage_types.append('1 ph')                
        if phase_indic == 2: #2 phase
                [phx,phy] = random.sample(active_phases, 2)
                Ckt_obj.dss.CktElement.Open(1,phx)
                Ckt_obj.dss.CktElement.Open(1,phy)
                outage_types.append('2 ph')  
        Ckt_obj.dss.Solution.Solve()

In [274]:
V_node_Sc = {}
for bus in node_list:
        V=Bus(Ckt_obj,bus).Vmag
        V_node_Sc[bus] = V    

In [275]:
flow_branch_Sc = {}  
for (u,v) in edge_list:
    branch_label = G_original[u][v]['Label']
    branch_device = G_original[u][v]['Device']
    branch_elem = f"{branch_device}.{branch_label}"
    branch_pflow = Branch(Ckt_obj,branch_elem).flow
    flow_branch_Sc[(u,v)] = np.sum(branch_pflow)

In [276]:
outage_edges

[('844', '846'), ('832', '858'), ('832', '888'), ('836', '862')]

In [277]:
outage_lines

['line.l22', 'line.l16', 'line.l20']

In [279]:
outage_types

['2 ph', '2 ph', '2 ph']

In [280]:
V_node_Sc

{'sourcebus': array([1.05000022, 1.04999967, 1.05000004]),
 '800': array([1.05000051, 1.04999981, 1.04999965]),
 '802': array([1.05004488, 1.04982231, 1.04998567]),
 '806': array([1.05006655, 1.04974932, 0.0724132 ]),
 '808': array([1.0504158 , 1.04912773, 0.07247626]),
 '810': array([0.        , 1.04897943, 0.        ]),
 '812': array([0.02652561, 0.00714505, 0.07249085]),
 '814': array([0.02653861, 0.00714329, 0.07248098]),
 '814r': array([0.02919243, 0.00785764, 0.07972919]),
 '850': array([0.02919244, 0.00785764, 0.07972918]),
 '816': array([0.02919255, 0.00785759, 0.07972886]),
 '818': array([0., 0., 0.]),
 '824': array([0.02919588, 0.00785589, 0.07971737]),
 '820': array([0., 0., 0.]),
 '822': array([0., 0., 0.]),
 '826': array([0.        , 0.00785435, 0.        ]),
 '828': array([0.02919616, 0.00785644, 0.07971674]),
 '830': array([0.02920703, 0.00787152, 0.07970672]),
 '854': array([0.02920715, 0.00787162, 0.07970706]),
 '832': array([0.03213093, 0.01104375, 0.00676045]),
 '858

In [281]:
flow_branch_Sc

{('sourcebus', '800'): 59.46431660363159,
 ('800', '802'): 59.464313090578365,
 ('802', '806'): 31.95611075804539,
 ('806', '808'): 16.887732875726066,
 ('808', '810'): 8.393004099452511,
 ('808', '812'): 0.08202166283638464,
 ('812', '814'): 0.08017256486087074,
 ('814', '814r'): 0.08097279246677098,
 ('814r', '850'): 0.08010300136690332,
 ('850', '816'): 0.08010297733043185,
 ('816', '818'): 2.508795615280279e-10,
 ('816', '824'): 0.08005078071481593,
 ('818', '820'): 0.0,
 ('824', '826'): 0.00123405777198968,
 ('824', '828'): 0.06479801511293615,
 ('820', '822'): 0.0,
 ('828', '830'): 0.04910343038618423,
 ('830', '854'): 0.0015436764580635275,
 ('854', '856'): 0.00012392085041104626,
 ('854', '852'): 0.0012955717501521343,
 ('832', '858'): 2.084547026251569e-09,
 ('832', '852r'): 0.0012908241271977702,
 ('832', '888'): 7.77638495523637e-10,
 ('858', '864'): 5.673610907301205e-138,
 ('858', '834'): -2.4905474562034577e-136,
 ('834', '860'): 9.940067879733732e-136,
 ('834', '842'): 2

In [285]:
nbus_sensors = math.ceil(len(node_list)*0.4)
nbranch_sensors = math.ceil(len(edge_list)*0.4)

In [289]:
bus_sensors = random.sample(node_list,nbus_sensors)

In [290]:
bus_sensors

['810',
 '808',
 '816',
 '848',
 '826',
 '854',
 '832',
 '850',
 '806',
 'sourcebus',
 '840',
 '846',
 '890',
 '862',
 '860']

In [291]:
branch_sensors = random.sample(edge_list,nbranch_sensors)

In [292]:
branch_sensors

[('860', '836'),
 ('814', '814r'),
 ('812', '814'),
 ('888', '890'),
 ('814r', '850'),
 ('828', '830'),
 ('862', '838'),
 ('830', '854'),
 ('808', '812'),
 ('820', '822'),
 ('834', '842'),
 ('sourcebus', '800'),
 ('842', '844'),
 ('824', '828'),
 ('844', '846')]

In [293]:
with open('bus_sensors.pkl', 'wb') as f:
    pickle.dump(bus_sensors, f)

In [294]:
with open('branch_sensors.pkl', 'wb') as f:
    pickle.dump(branch_sensors, f)